In [1]:
from sklearn.preprocessing import MinMaxScaler
import numpy as np

base = np.array([[20],[30],[50],[120],[22],[63],[102],[34],[87],[12],[25],[128],[23],[77],[56]])
target = np.array([[50],[70],[110],[250],[54],[136],[214],[78],[184],[34],[60],[266],[56],[166],[122]])
#mixed = base
#mixed = base.extend(target)
mixed = np.concatenate((base,target))

scaler = MinMaxScaler()
mixed_scaled = scaler.fit_transform(mixed)

base_scaled = mixed_scaled[:15]
#base_unscaled = scaler.inverse_transform(base_scaled)

target_scaled = mixed_scaled[15:20]
#target_unscaled = scaler.inverse_transform(target_scaled)
#print(mixed)
# scaler = MinMaxScaler()
# base_scaled = scaler.fit_transform(base)
# base_unscaled = scaler.inverse_transform(base_scaled)

# target_scaled = scaler.fit_transform(target[:5])
# target_unscaled = scaler.inverse_transform(target_scaled)

num_clusters = 3
num_attr = mixed_scaled[0].size
print(target_scaled)

[[ 0.1496063 ]
 [ 0.22834646]
 [ 0.38582677]
 [ 0.93700787]
 [ 0.16535433]]


/home/tonmoy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, _DataConversionWarning)


In [23]:
from sklearn import preprocessing
import numpy as np

base = np.array([[20],[30],[50],[120],[22],[63],[102],[34],[87],[12],[25],[128],[23],[77],[56]])
target = np.array([[50],[70],[110],[250],[54],[136],[214],[78],[184],[34],[60],[266],[56],[166],[122]])


base_scaled = preprocessing.scale(base)

#base_unscaled = scaler.inverse_transform(base_scaled)

target_scaled = preprocessing.scale(target)
target_scaled = target_scaled[:5]
num_clusters = 3
num_attr = base_scaled[0].size

print(base_scaled)
print("****")
print(target_scaled)

Normalizer(copy=True, norm='l2')
[[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]]
****
[[ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]
 [ 1.]]


/home/tonmoy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


In [26]:
from sklearn import preprocessing
import numpy as np

base = np.array([[20],[30],[50],[120],[22],[63],[102],[34],[87],[12],[25],[128],[23],[77],[56]])
target = np.array([[50],[70],[110],[250],[54],[136],[214],[78],[184],[34],[60],[266],[56],[166],[122]])


scaler = preprocessing.StandardScaler().fit(base)

base_scaled = scaler.transform(base)

#base_unscaled = scaler.inverse_transform(base_scaled)

target_scaled = scaler.transform(target)
target_scaled = target_scaled[:5]
num_clusters = 3
num_attr = base_scaled[0].size

print(base_scaled)
print("****")
print(target_scaled)

[[-0.99062298]
 [-0.71996096]
 [-0.17863693]
 [ 1.71599718]
 [-0.93649057]
 [ 0.17322369]
 [ 1.22880555]
 [-0.61169615]
 [ 0.82281253]
 [-1.20715259]
 [-0.85529197]
 [ 1.93252679]
 [-0.90942437]
 [ 0.55215051]
 [-0.01623972]]
****
[[-0.17863693]
 [ 0.3626871 ]
 [ 1.44533516]
 [ 5.23460338]
 [-0.07037212]]


/home/tonmoy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [27]:

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=num_clusters)
kmeans.fit(base_scaled)

base_centroids = kmeans.cluster_centers_
#labels = kmeans.labels_
    
print(kmeans.cluster_centers_)
print(kmeans.inertia_)

[[ 0.27066202]
 [ 1.62577651]
 [-0.89009137]]
1.15842410339


In [28]:
def cal_centroids(alpha,beta):

    target_centroids = []
    for i in range(num_clusters):
        val = []
        for j in range(num_attr):
            val.append(alpha[j]*base_centroids[i][j] + beta[j])
        
        target_centroids.append(val)
             

    target_centroids = np.array(target_centroids)
    print("new centroids: ", target_centroids)
    return target_centroids
    #print(target_centroids)

In [29]:
def kmeans(k,dataItems,centroids,maxIter,num_attr):
    
    #print(k,dataItems,centroids,maxInter,num_attr)
    old_centroids = []
    groups = []
    
    for i in range(k):
        groups.append([])
        
    iter = 0
    while(iter < maxIter):
        old_centroids = centroids
        for item in dataItems:
            row = []
            for centroid in centroids:
                diff = 0
                for i in range(num_attr):
                    diff += abs(item[i] - centroid[i])
                    
                row.append(diff)
                
            idx = row.index(min(row))
            groups[idx].append(item)
                    
            
        iter += 1
        
    ss = 0    
    for i in range(k):
        for item in groups[i]:
            for j in range(num_attr):
            
                diff = abs(item[j]- old_centroids[i][j])
                ss += pow(diff,2)
            
        
    return (groups,ss/2)
        

In [30]:
def cal_gradient(clusters,reframed_centroids,old_alpha,old_beta):
    
    gradient_alpha = []
    gradient_beta = []
    gradient = 0
    
    for i in range(num_attr):
        gradient_alpha.append(0)
        gradient_beta.append(0)
        
    
    for i in range(num_clusters):
                
        for member in clusters[i]:
                      
            for j in range(num_attr):
                
                gradient =  (reframed_centroids[i][j] - member[j])
                gradient_alpha[j] = gradient_alpha[j] + gradient*reframed_centroids[i][j]
                gradient_beta[j] = gradient_beta[j] + gradient

    print("******")
    #print(gradient_alpha,gradient_beta)
    
    new_alpha = []
    new_beta = []
    
    for i in range(num_attr):       
        new_alpha.append(old_alpha[i]-.1*gradient_alpha[i])
        new_beta.append(old_beta[i]-.1*gradient_beta[i])
        
    return [new_alpha,new_beta]

In [31]:
alpha = []
beta = []
avg_m = []
avg_d = []
sum_d = 0
for i in range(num_attr):
    sum_m = 0
    for j in range(num_clusters):
        sum_m = sum_m + base_centroids[j][i]
        
    avg_m.append(sum_m/num_clusters)
    

for i in range(num_attr):
    sum_d = 0
    for j in range(len(target_scaled)):
        sum_d = sum_d + target_scaled[j][i]
        
    avg_d.append(sum_d/len(target_scaled))
    

for i in range(num_attr):
    alpha.append(avg_m[i]/avg_d[i])
    beta.append(0)

print(alpha,beta)
        


[0.24688547397710736] [0]


In [32]:
reframed_centroids =  cal_centroids(alpha,beta)
km = kmeans(num_clusters, target_scaled, reframed_centroids, 1, 1)
#kmeans.fit(target_scaled)
best_error = km[1]
#centroids = kmeans.cluster_centers_
#labels = kmeans.labels_

count = 0
best_alpha = alpha
best_beta = beta
while(1):
    #print(centroids)
    print("best error: ",best_error)
    
    #clusters = find_members(centroids,labels)
    
    #reframed_centroids = closest_centroids(reframed_centroids,centroids)
    
    new_alphabeta = cal_gradient(km[0],reframed_centroids,alpha,beta)
    
    alpha = new_alphabeta[0]
    beta = new_alphabeta[1]
    
    print("new alpha beta", alpha, beta)
    
    reframed_centroids =  cal_centroids(alpha,beta)
    
    km = kmeans(num_clusters, target_scaled, reframed_centroids, 1, 1)
    #kmeans.fit(target_scaled)
    new_error = km[1]
    
    print("compare ",best_error,new_error)
    if(new_error < best_error):
        best_alpha = alpha
        best_beta = beta
        best_error = new_error
        count = 0
        
    elif(new_error == best_error):
        if(count<5):
            count += 1
            continue
        else:
            break;
                
    else:
        break; 

    #base_centroids = kmeans.cluster_centers_
    #labels = kmeans.labels_

    #print(kmeans.cluster_centers_)
    #print(old_error,new_error)
    
print("finalparameters", best_alpha,best_beta)

new centroids:  [[ 0.06682252]
 [ 0.4013806 ]
 [-0.21975063]]
best error:  12.2359466899
******
new alpha beta [0.47941064462222993] [0.57424028839812802]
new centroids:  [[ 0.70399854]
 [ 1.35365485]
 [ 0.14752101]]
compare  12.2359466899 7.63515979818
best error:  7.63515979818
******
new alpha beta [1.0123157293615539] [0.93861467332664672]
new centroids:  [[ 1.21261009]
 [ 2.5844138 ]
 [ 0.03756118]]
compare  7.63515979818 3.62088190988
best error:  3.62088190988
******
new alpha beta [1.7254585930744577] [1.2270055889812788]
new centroids:  [[ 1.69402169]
 [ 4.03221563]
 [-0.30881021]]
compare  3.62088190988 1.01614386477
best error:  1.01614386477
******
new alpha beta [2.1360396031660613] [1.4263865797003876]
new centroids:  [[ 2.00453136]
 [ 4.89910958]
 [-0.47488384]]
compare  1.01614386477 0.689086755602
best error:  0.689086755602
******
new alpha beta [2.1152562727236162] [1.5578492948914475]
new centroids:  [[ 2.13036882]
 [ 4.99678325]
 [-0.32492206]]
compare  0.689086755

In [33]:
target_cent = cal_centroids(best_alpha,best_beta)

new centroids:  [[ 2.2014446 ]
 [ 4.88575493]
 [-0.09786098]]


In [34]:
target_scaled = scaler.transform(target)
# base_unscaled = scaler.inverse_transform(base_scaled)
# target_unscaled = scaler.inverse_transform(target_sc)
# print(target_sc)
# print(base_scaled)
# print(base_unscaled)
# print(target_unscaled)

/home/tonmoy/anaconda3/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, _DataConversionWarning)


In [35]:
kmeans = KMeans(n_clusters=num_clusters, init=target_cent, max_iter=1)
kmeans.fit(target_scaled)
print(kmeans.inertia_)

4.69699126207


/home/tonmoy/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)


In [36]:
kmeans = KMeans(n_clusters=num_clusters, init=base_centroids, max_iter=1)
kmeans.fit(target_scaled)
print(kmeans.inertia_)

15.9327036408


/home/tonmoy/anaconda3/lib/python3.6/site-packages/sklearn/cluster/k_means_.py:889: RuntimeWarning: Explicit initial center position passed: performing only one init in k-means instead of n_init=10
  return_n_iter=True)
